# 05 – Data Analysis & Feature Engineering

In [ ]:
from __future__ import annotations

import math
import os
import time

import h5py
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import pearsonr
from src.config import get_dataset_path
from src.utils.feature_engineering import layer_dataframe
from src.utils.part_anomaly import part_anomaly_fractions
from tqdm.auto import tqdm

sns.set_context('talk')
sns.set_style('darkgrid')

The **build_features.py** file essentially converts a **.hdf5** file of a build into a **.csv** one.

In [ ]:
# !python -m src.utils.build_features \
#        --h5 "../data/2021-04-16 TCR Phase 1 Build 2.hdf5" \
#        --out "../data/build2_features.csv"

In [ ]:
df = pd.read_csv("../data/build2_features.csv")

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
df_layer = df[["layer", "gas_oxygen", "top_flow_rate", "spatter_px", "streak_px"]]

In [ ]:
df_layer.tail()

### 1. Oxygen spikes vs next‑layer spatter

In [ ]:
# Shift O₂ up by one layer so current‑layer oxygen predicts next‑layer spatter
df_q1 = (
    df_layer
    .assign(gas_oxygen_prev=df_layer["gas_oxygen"].shift(0),
            spatter_next   =df_layer["spatter_px"].shift(-1))
    .dropna()
)

r_oxy, p_oxy = pearsonr(df_q1["gas_oxygen_prev"], df_q1["spatter_next"])
title_txt = (
    "O₂ level (layer L)  vs  Spatter pixels (layer L+1)\n"
    f"Pearson r = {r_oxy:.3f},   p = {p_oxy:.3g}"
)

plt.figure(figsize=(12, 5))
sns.regplot(data=df_q1, x="gas_oxygen_prev", y="spatter_next",
            scatter_kws={"s": 4}, truncate=True)
plt.title(title_txt)
plt.xlabel("Gas-loop O₂ [ppm]")
plt.ylabel("Spatter pixel count (next layer)")
plt.tight_layout()
plt.show()

### 2. Low top‑flow rates vs recoater streaking

In [ ]:
df_tmp = df_layer[["top_flow_rate", "streak_px"]].dropna()

r_flow, p_flow = pearsonr(df_tmp["top_flow_rate"], df_tmp["streak_px"])

plt.figure(figsize=(12, 5))
sns.regplot(
    data=df_tmp, x="top_flow_rate", y="streak_px",
    scatter_kws={"s": 4}, truncate=True
)
plt.title(
    "Top flow rate  vs  Recoater-streak pixels\n"
    f"Pearson r = {r_flow:.3f},   p = {p_flow:.3g}"
)
plt.xlabel("Top flow rate [L min⁻¹]")
plt.ylabel("Recoater streak pixel count")
plt.tight_layout()
plt.show()


### 3. Parts with lowest anomaly fraction

In [ ]:
with h5py.File(DATA_FILE, "r") as h5:
    df_parts = part_anomaly_fractions(h5)

df_parts.head()

### 4. Tall part anomaly trend vs build height

In [ ]:
TALL_PARTS       = df_parts.index[-5:]          # or pick any list of PIDs
layers_per_chunk = 16                           # match dataset.chunks[0] if possible


with h5py.File(DATA_FILE, "r") as h5:
    ds_part = h5["slices/part_ids"]
    ds_spat = h5["slices/segmentation_results/8"]

    n_layers, H, W = ds_part.shape
    P               = len(TALL_PARTS)
    tall_pids       = np.asarray(TALL_PARTS, dtype=ds_part.dtype)

    # output: rows = layer, cols = tall part IDs
    layer_spatter = np.zeros((n_layers, P), dtype=np.int64)

    # reusable read buffers (one HDF5 chunk)
    part_buf = np.empty((layers_per_chunk, H, W), dtype=ds_part.dtype)
    spat_buf = np.empty_like(part_buf, dtype=bool)

    rng = tqdm(range(0, n_layers, layers_per_chunk),
               total=(n_layers + layers_per_chunk - 1) // layers_per_chunk,
               desc="Streaming layers", unit="blk")

    for start in rng:
        stop   = min(start + layers_per_chunk, n_layers)
        sl_out = slice(0, stop - start)               # length in this chunk

        ds_part.read_direct(part_buf, source_sel=slice(start, stop), dest_sel=sl_out)
        ds_spat.read_direct(spat_buf, source_sel=slice(start, stop), dest_sel=sl_out)

        # Broadcast compare: (P, L, H, W)  boolean
        match_pid = (part_buf[np.newaxis, :sl_out.stop] == tall_pids[:, None, None, None])
        # Logical AND with spatter mask, then sum pixels → shape (P, L)
        counts = np.logical_and(match_pid, spat_buf[np.newaxis, :sl_out.stop]).sum(axis=(2, 3))

        # counts.T is (L, P) so we can drop straight into the big table
        layer_spatter[start:stop] += counts.T

In [ ]:
df_tall = pd.DataFrame(layer_spatter, columns=TALL_PARTS)
df_cum  = df_tall.cumsum()

In [ ]:
plt.figure(figsize=(10, 5))
for pid in TALL_PARTS:
    plt.plot(df_cum.index, df_cum[pid], label=f"Part {pid}")
plt.xlabel("Layer number")
plt.ylabel("Cumulative spatter pixels [count]")
plt.title("Cumulative spatter vs build height (tall parts)")
plt.legend()
plt.tight_layout()
plt.show()


### 5. Laser power / speed vs spatter occurrence

In [ ]:
with h5py.File(DATA_FILE, "r") as h5:
    laser_power = h5["parts/process_parameters/laser_beam_power"][:]
    laser_speed = h5["parts/process_parameters/laser_beam_speed"][:]

df_params = pd.DataFrame(
    {
        "anomaly_frac": df_parts["anomaly_frac"],
        "laser_power":  laser_power[df_parts.index - 1],
        "laser_speed":  laser_speed[df_parts.index - 1],
    },
    index=df_parts.index,
)

r_pwr, p_pwr = pearsonr(df_params["laser_power"], df_params["anomaly_frac"])
r_spd, p_spd = pearsonr(df_params["laser_speed"], df_params["anomaly_frac"])

fig, ax = plt.subplots(1, 2, figsize=(12, 5), sharey=True)

sns.regplot(
    data=df_params, x="laser_power", y="anomaly_frac",
    truncate=True, scatter_kws={"s": 25}, ax=ax[0]
)
ax[0].set(
    xlabel="Laser power [W]",
    ylabel="Anomaly fraction [–]",
    ylim=(-1, 1),
    title=f"Anomaly fraction vs laser power\n"
          f"Pearson r = {r_pwr:.3f},  p = {p_pwr:.3g}",
)

sns.regplot(
    data=df_params, x="laser_speed", y="anomaly_frac",
    truncate=True, scatter_kws={"s": 25}, ax=ax[1]
)
ax[1].set(
    xlabel="Scan speed [mm s$^{-1}$]",
    ylim=(-1, 1),
    title=f"Anomaly fraction vs scan speed\n"
          f"Pearson r = {r_spd:.3f},  p = {p_spd:.3g}",
)

plt.tight_layout()


In [ ]:
# sanity: compare 3 random parts
for pid in np.random.choice(df_parts.index, 3, replace=False):
    print(pid, laser_power[pid-1], laser_speed[pid-1])

### 6. Recoater damage onset vs streak progression

In [ ]:
# TODO: check if recoater‑health signal available, align with streak counts
df_layer['streak_cumsum'] = df_layer['streak_px'].cumsum()
df_layer[['layer', 'streak_px', 'streak_cumsum']].tail()

## Physics Informed Extensions 

### 7. Predict *Under‑* and *Over‑Melting Area from Process Parameters

Goal: Use only per‑part process parameters to estimate the fraction of surface classified as under melting or over melting.

In [ ]:
CLASS_ID_UNDER = 6
CLASS_ID_OVER  = 7
TARGET_RAM_MB  = 200                                  # keep buffers under this

with h5py.File(DATA_FILE, "r") as h5:
    ds_part  = h5["slices/part_ids"]
    ds_under = h5[f"slices/segmentation_results/{CLASS_ID_UNDER}"]
    ds_over  = h5[f"slices/segmentation_results/{CLASS_ID_OVER}"]

    nL, H, W = ds_part.shape

    bytes_per_px = ds_part.dtype.itemsize + 2  # uint32 + two bool bytes
    max_layers   = max(1, (TARGET_RAM_MB * 1024**2) // (bytes_per_px * H * W))
    chunk_L      = min(ds_part.chunks[0] if ds_part.chunks else 16, max_layers)

    max_pid_guess = int(ds_part.attrs.get("max_part_id", 0)) or int(ds_part[:chunk_L].max())
    total_px = np.zeros(max_pid_guess + 1, dtype=np.int64)
    under_px = np.zeros_like(total_px)
    over_px  = np.zeros_like(total_px)

    part_buf  = np.empty((chunk_L, H, W), dtype=ds_part.dtype)
    under_buf = np.empty_like(part_buf, dtype=bool)
    over_buf  = np.empty_like(part_buf,  dtype=bool)

    for start in tqdm(range(0, nL, chunk_L), desc="Scanning", unit="blk"):
        stop   = min(start + chunk_L, nL)
        n_here = stop - start
        sl_out = slice(0, n_here)

        ds_part.read_direct( part_buf,  slice(start, stop), sl_out)
        ds_under.read_direct(under_buf, slice(start, stop), sl_out)
        ds_over.read_direct( over_buf,  slice(start, stop), sl_out)

        part_flat = part_buf[:n_here].ravel()          # view, no copy

        total_px  += np.bincount(part_flat, minlength=total_px.size)

        counts = np.bincount(
            part_flat,
            weights=under_buf[:n_here].ravel().astype(np.uint8),
            minlength=total_px.size,
        )
        under_px += counts.astype(np.int64)
        over_px  += np.bincount(
            part_flat,
            weights=over_buf[:n_here].ravel().astype(np.uint8),
            minlength=total_px.size,
        ).astype(np.int64)

In [ ]:
valid = np.flatnonzero(total_px)
valid = valid[valid != 0]                      # drop background

df_melt = pd.DataFrame(
    {
        "under_frac": under_px[valid] / total_px[valid],
        "over_frac": over_px[valid] / total_px[valid],
    },
    index=valid,
)

In [ ]:
with h5py.File(DATA_FILE, "r") as h5:
    power = h5["parts/process_parameters/laser_beam_power"][:]
    speed = h5["parts/process_parameters/laser_beam_speed"][:]
    hatch = h5["parts/process_parameters/hatch_spacing"][:]

df_melt["power"] = power[df_melt.index - 1]     # 1-based IDs
df_melt["speed"] = speed[df_melt.index - 1]
df_melt["hatch"] = hatch[df_melt.index - 1]

df_melt.head()

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

X = df_melt[['power', 'speed', 'hatch']]
y_under = df_melt['under_frac']
y_over  = df_melt['over_frac']

model = LinearRegression()
mae_under = -cross_val_score(model, X, y_under, cv=5, scoring='neg_mean_absolute_error').mean()
mae_over  = -cross_val_score(model, X, y_over,  cv=5, scoring='neg_mean_absolute_error').mean()

print(f'MAE (under‑melt frac) = {mae_under:.4f}')
print(f'MAE (over‑melt  frac) = {mae_over:.4f}')

#### Visualising Response Surfaces
Below we create 2‑D projections to see where the linear model (or raw data) suggests
minimal melt‑pool anomalies.

In [ ]:
# Heatmap of anomaly vs power & hatch spacing
import matplotlib.tri as mtri

triang = mtri.Triangulation(df_melt['power'], df_melt['hatch'])

fig, axes = plt.subplots(1, 2, figsize=(12, 5), sharex=True, sharey=True)

for ax, target, title in zip(
    axes,
    ['under_frac', 'over_frac'],
    ['Under‑melt fraction', 'Over‑melt fraction'],
    strict=False
):
    tpc = ax.tricontourf(triang, df_melt[target], levels=14)
    fig.colorbar(tpc, ax=ax)
    ax.set_title(title)
    ax.set_xlabel('Laser power [W]')
    ax.set_ylabel('Hatch spacing [µm]')

fig.suptitle('Anomaly fraction vs (Power, Hatch Spacing)')
plt.tight_layout()


## 8. Physics‑Informed Analytics

This section implements **first‑order physics models** directly on the build logs and
segmentation outputs so we can test whether simple analytical theory aligns with observed anomalies.

> **Disclaimer** &nbsp; All constants are ballpark values for 316 L steel and a Concept Laser M2‑class
> machine. Adjust as needed for your exact material & machine.

In [ ]:
# Physical constants 
R_GAS = 8.314            # J·mol⁻¹·K⁻¹
LAT_VAP = 6e6            # J·kg⁻¹  (approx latent heat of vaporisation)
A_RECOIL = 1e5           # Pa – empirical scaling
k_COND = 16              # W·m⁻¹·K⁻¹ thermal conductivity (solid, 316L)
ALPHA = 4e-6             # m²·s⁻¹ thermal diffusivity
LASER_RADIUS = 50e-6     # m (spot radius)
ABSORPTIVITY = 0.35      # fraction of P coupled to substrate
T_AMBIENT = 300          # K

# Gas properties 
rho_Ar = 1.6             # kg·m⁻³ at ~100 °C
rho_powder = 7800        # kg·m⁻³ (solid density proxy)
NOZZLE_AREA = 1.0e-3     # m² (placeholder cross‑section)

# Recoater properties 
L_BLADE = 0.25           # m (blade length)
E_BLADE = 200e9          # Pa (steel modulus)
I_BLADE = 1.5e-8         # m⁴ (estimated rectangular section)
LAYER_THICKNESS = 50e-6  # m

def rosenthal_Tmax(P: float, v: float, r: float = LASER_RADIUS) -> float:
    """Return peak melt‑pool surface temperature using a 2‑D Rosenthal approximation."""
    Q = ABSORPTIVITY * P  # effective absorbed power (W)
    deltaT = Q / (2 * math.pi * k_COND * r) * math.exp(-v * r / (2 * ALPHA))
    return T_AMBIENT + deltaT


def recoil_pressure(T_surf: float) -> float:
    """Approximate recoil pressure from Clausius–Clapeyron‑like exponential."""
    return A_RECOIL * math.exp(-LAT_VAP / (R_GAS * T_surf))


def plume_velocity(Q_l_min: float) -> float:
    """Argon volumetric flow `Q` (l/min) → plume advective velocity (m/s)."""
    Q_m3_s = Q_l_min / 1000 / 60  # convert to m³/s
    return Q_m3_s / NOZZLE_AREA * (1 - rho_powder / rho_Ar)


def blade_deflection(F_tip: float) -> float:
    """Tip deflection of a cantilever blade with end load F."""
    return F_tip * L_BLADE**3 / (3 * E_BLADE * I_BLADE)


### 8.1 Melt‑Pool Physics → Spatter Correlation

In [ ]:
# Compute per-part recoil pressure & correlate with anomaly fraction
df_phys = df_melt.copy()

# Directly merge anomaly_frac from df_params based on laser power and speed matching
df_phys = pd.merge(df_phys, df_params, left_on=['power', 'speed'], right_on=['laser_power', 'laser_speed'], how='left')

T_max_arr, P_recoil_arr = [], []
for idx, row in df_phys.iterrows():
    P, v = row['power'], row['speed']
    Tmax = rosenthal_Tmax(P, v)
    P_rec = recoil_pressure(Tmax)
    T_max_arr.append(Tmax)
    P_recoil_arr.append(P_rec)

df_phys['Tmax_K'] = T_max_arr
df_phys['P_recoil_Pa'] = P_recoil_arr
df_phys['high_recoil'] = df_phys['P_recoil_Pa'] > 1.0e5  # P_crit = 1 bar

print(df_phys[['power', 'speed', 'Tmax_K', 'P_recoil_Pa', 'anomaly_frac']].head())

# Visualization
plt.figure(figsize=(6, 5))
sns.boxplot(x='high_recoil', y='anomaly_frac', data=df_phys)
plt.xlabel('P_recoil > 1 bar?')
plt.ylabel('Spatter+Streak anomaly fraction')
plt.title('High recoil pressure vs anomaly fraction')
plt.tight_layout()

In [ ]:
df_phys.head()

### 8.2 Gas‑Flow Plume Velocity vs Spatter Dispersion

In [ ]:
# Approximate plume velocity per layer 
df_layer['plume_v_ms'] = df_layer['top_flow_rate'].apply(plume_velocity)

corr_plume, p_value = pearsonr(df_layer['plume_v_ms'], df_layer['spatter_px'])
print(f'Pearson r(plume_v, spatter_px) = {corr_plume:.3f}, p-value = {p_value:.3e}')

plt.figure(figsize=(6, 5))
sns.regplot(x='plume_v_ms', y='spatter_px', data=df_layer, scatter_kws={'s': 4})
plt.xlabel('Plume advective velocity [m/s]')
plt.ylabel('Spatter pixel count')
plt.title(f'Gas plume vs spatter\nPearson r = {corr_plume:.3f}, p-value = {p_value:.3e}')

# Shorten axis values to scientific notation
plt.ticklabel_format(axis='both', style='sci', scilimits=(0, 0))

plt.tight_layout()

### 8.3 Recoater Blade Deflection vs Streak Onset

In [ ]:
# Use anomaly‑height proxy: streak pixels translate to tip load
LOAD_PER_PX = 1e-3  # N per streak pixel (placeholder scaling)

df_layer['tip_load_N'] = df_layer['streak_px'] * LOAD_PER_PX
df_layer['blade_defl_m'] = df_layer['tip_load_N'].apply(blade_deflection)
df_layer['exceeds_powder'] = df_layer['blade_defl_m'] > LAYER_THICKNESS

prop_exceed = df_layer['exceeds_powder'].mean()
print(f'%.1f%% of layers exceed powder thickness deflection' % (100*prop_exceed))

sns.lineplot(x='layer', y='blade_defl_m', data=df_layer[::50])
plt.axhline(LAYER_THICKNESS, c='r', ls='--', label='Powder layer height')
plt.ylabel('Blade tip deflection [m]')
plt.title('Recoater deflection across build')
plt.legend();
plt.tight_layout()

### 8.4 Additional Metrics & Correlations

In [ ]:
# Melt‑pool aspect ratio proxy 
df_phys['energy_density'] = df_phys['power'] / (df_phys['speed'] * df_phys['hatch'] * 50e-6)
df_phys['aspect_ratio'] = 0.8 * np.sqrt(df_phys['energy_density']) / (0.2 * df_phys['energy_density'])

corr_ar_under = df_phys['aspect_ratio'].corr(df_phys['under_frac'])
corr_ar_over  = df_phys['aspect_ratio'].corr(df_phys['over_frac'])
print(f'Aspect ratio vs under‑melt r = {corr_ar_under:.3f}')
print(f'Aspect ratio vs over‑melt  r = {corr_ar_over :.3f}')

# Cumulative energy density vs streak onset 
cum_ED = df_layer['spatter_px'].cumsum()  # proxy; replace with ED/time integral if logs exist
sns.lineplot(x=df_layer['layer'], y=cum_ED)
plt.xlabel('Layer')
plt.ylabel('Cumulative spatter pixels (proxy ED)')
plt.title('Cumulative energy proxy vs build height')
plt.tight_layout()


### 9. Feature Selection

In [ ]:
# df_selected.to_csv('../data/selected_features.csv', index=False)